# Lecture 10

From previous lecture:

In [1]:
# Import libraries we will use
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from collections import OrderedDict

## Base Classes

In [2]:
# Create some virtual classes
class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)


## Grade Representation


In [3]:
class grade(data):
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None):
        self.__value=value
        self.__numerical=numerical
        self.__gradebook_name=str()
        
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
        self.__gradebook_name=name
        data.__init__(self,name+" Grade Data Object")        

    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print( self.name()+" Error: Bad Grade.")
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)


## Student Representation

In [4]:
class student(data):
    def __init__(self, first_name, last_name, id_number):
        self.__grades=dict()
        self.__id_number=id_number
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print (self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists.")
            raise Exception

    def id_number(self):
        return self.__id_number
    
    def grade_names(self):
        return self.__grades.keys()
    
    def grades(self):
        return self.__grades
    
    def __getitem__(self,key):
        return self.__grades[key]
    
    def print_grades(self):
        for grade in self.__grades:
            print (self.__grades[grade])

## Grade Calculator

In [5]:
class grade_calculator(alg):    
    def __init__(self,name,stats):
        self.__stats=stats
        alg.__init__(self,name)

    def apply(self,a_grade):
        raise NotImplementedError
        

class uncurved_letter_grade_percent(grade_calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        grade_calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade),
                                 False)
        

    def apply(self,a_grade):
        if not isinstance(a_grade,grade):
            print (self.name()+ " Error: Did not get an proper grade as input.")
            raise Exception
        if not a_grade.numerical():
            print (self.name()+ " Error: Did not get a numerical grade as input.")
            raise Exception

        percent=a_grade.value()/self.__max_grade
        
        for i,v in enumerate(self.__grades_definition):
            if percent>=v[0]:
                break
                            
        return grade(self.__grade_name,value=self.__grades_definition[i][1])
            

In [6]:
class curved_letter_grade(grade_calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        grade_calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade),
                                   False)
        

    def apply(self,a_grade):
        if not isinstance(a_grade,grade):
            print (self.name()+ " Error: Did not get an proper grade as input.")
            raise Exception
        if not a_grade.numerical():
            print (self.name()+ " Error: Did not get a numerical grade as input.")
            raise Exception
    
        # Rescale the grade
        percent=a_grade.value()/self.__max_grade
        shift_to_zero=percent-(self.__mean/self.__max_grade)
        scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
        scaled_percent=scale_std+0.8
        
        for i,v in enumerate(self.__grades_definition):
            if scaled_percent>=v[0]:
                break
                            
        return grade(self.__grade_name,value=self.__grades_definition[i][1])
            

## Stats Computation

In [7]:
import numpy as np
import math

class statistics_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,grades):
        raise NotImplementedError
        
class mean_std_calculator(statistics_calculator):
    def __init__(self):
        statistics_calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,grades):
        return np.mean(grades),math.sqrt(np.var(grades))



## Grade Summing

In [8]:
class summary_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_student):
        raise NotImplementedError
        
class grade_summer(summary_calculator):
    def __init__(self,prefix,n):
        self.__prefix=prefix
        self.__n=n
        statistics_calculator.__init__(self,"Sum Grades")
        
    def apply(self,a_student):
        labels=[self.__prefix+str(x) for x in range(1,self.__n)]
        
        grade_sum=0.
        for label in labels:
            grade_sum+=a_student[label].value()
            
        return grade(self.__prefix+"sum",value=grade_sum)


## Gradebook

In [9]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class 
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()        
            
    # New method to add data        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    # New method to allow iterating over students
    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
    
    def apply_summary(self,a_grader):
        for k,a_student in self.__students.items():
            a_student.add_grade(a_grader.apply(a_student))
    
    def apply_grader(self,a_grader,grade_name):
        for k,a_student in self.__students.items():
            a_student.add_grade(a_grader.apply(a_student[grade_name]))
            
    def apply_stats(self,a_stat_comp,grade_name):
        grades=list()
        for k,a_student in self.__students.items():
            grades.append(a_student[grade_name].value())
        return a_stat_comp.apply(grades)
                
    def students(self):
        return self.__students
            
    def print_grades(self,grade_name):
        if isinstance(grade_name,str):
            grade_names=list()
            grade_names.append(grade_name)
        else:
            grade_names=grade_name
                      
        for k,a_student in self.__students.items():
            print (a_student.name(),end="")
            for a_grade_name in grade_names:
                print (a_student[a_grade_name],end="")
            print()
            
    def print_students(self):    
        for k,a_student in self.__students.items():
            print (k, a_student.name())
            a_student.print_grades()
            print ("_______________________________________")
            

## Building a Gradebook

In [10]:
# Read Data into a Pandas DataFrame
df = pd.read_csv("../Lecture.9/Data-1401-Grades-Fixed.csv")

In [11]:
# Create mask to keep only lines with numbers
mask=list()
for i in range(16):
    mask.append(True)
    mask.append(False)
    
# Apply mask and remove NaNs
df_0=df[mask].fillna(0)

# Fix Exam 1 entries
df_0["Exam 1 Fixed"] = list(map(lambda x: int(x.split("-")[0]) ,df_0["Exam 1"].tolist()))

In [12]:
df_0.keys()

Index(['Lab 1', 'Lab 2', 'Lab 3', 'Lab 4', 'Exam 1', 'Lab 5', 'Unnamed: 6',
       'Lab 7', 'Exam 2', 'Exam 1 Fixed'],
      dtype='object')

In [13]:
def build_grade_book(df_0):
    a_grade_book=grade_book("Data 1401")

    for student_i in range(df_0.shape[0]):
        a_student_0=student("Student",str(student_i),student_i)

        # Add data
        for k in df_0.keys():
            try:
                a_student_0.add_grade(grade(k,value=float(df_0[k].tolist()[student_i])))
            except:
                a_student_0.add_grade(grade(k,value=str(df_0[k].tolist()[student_i])))


        a_grade_book.add_student(a_student_0)

    return a_grade_book
        

In [15]:
a_grade_book=build_grade_book(df_0)

In [16]:
a_grade_book.print_students()

0 Student 0 Student Data
Lab 1: 0.0
Lab 2: 0.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 0.0
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 0.0
_______________________________________
1 Student 1 Student Data
Lab 1: 100.0
Lab 2: 96.0
Lab 3: 46.0
Lab 4: 0.0
Exam 1: 68-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 68.0
_______________________________________
2 Student 2 Student Data
Lab 1: 100.0
Lab 2: 100.0
Lab 3: 90.0
Lab 4: 0.0
Exam 1: 88-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 88.0
_______________________________________
3 Student 3 Student Data
Lab 1: 93.0
Lab 2: 100.0
Lab 3: 24.0
Lab 4: 0.0
Exam 1: 96-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 96.0
_______________________________________
4 Student 4 Student Data
Lab 1: 0.0
Lab 2: 81.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 43-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 43.0
_______________________________________
5 Student 5 Student D

In [17]:
Lab_2_stats=a_grade_book.apply_stats(mean_std_calculator(),"Lab 2")
print(Lab_2_stats)

(85.8125, 23.688654325436048)


In [18]:
a_grade_book.apply_grader(curved_letter_grade("Lab 2 Letter",Lab_2_stats[0],Lab_2_stats[1]), "Lab 2")

In [19]:
a_grade_book.print_students()

0 Student 0 Student Data
Lab 1: 0.0
Lab 2: 0.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 0.0
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 0.0
Lab 2 Letter: F-
_______________________________________
1 Student 1 Student Data
Lab 1: 100.0
Lab 2: 96.0
Lab 3: 46.0
Lab 4: 0.0
Exam 1: 68-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 68.0
Lab 2 Letter: B
_______________________________________
2 Student 2 Student Data
Lab 1: 100.0
Lab 2: 100.0
Lab 3: 90.0
Lab 4: 0.0
Exam 1: 88-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 88.0
Lab 2 Letter: B
_______________________________________
3 Student 3 Student Data
Lab 1: 93.0
Lab 2: 100.0
Lab 3: 24.0
Lab 4: 0.0
Exam 1: 96-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 96.0
Lab 2 Letter: B
_______________________________________
4 Student 4 Student Data
Lab 1: 0.0
Lab 2: 81.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 43-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 4

In [20]:
a_grade_book.apply_summary(grade_summer("Lab ",5))

In [21]:
a_grade_book.print_students()

0 Student 0 Student Data
Lab 1: 0.0
Lab 2: 0.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 0.0
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 0.0
Lab 2 Letter: F-
Lab sum: 0.0
_______________________________________
1 Student 1 Student Data
Lab 1: 100.0
Lab 2: 96.0
Lab 3: 46.0
Lab 4: 0.0
Exam 1: 68-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 68.0
Lab 2 Letter: B
Lab sum: 242.0
_______________________________________
2 Student 2 Student Data
Lab 1: 100.0
Lab 2: 100.0
Lab 3: 90.0
Lab 4: 0.0
Exam 1: 88-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 88.0
Lab 2 Letter: B
Lab sum: 290.0
_______________________________________
3 Student 3 Student Data
Lab 1: 93.0
Lab 2: 100.0
Lab 3: 24.0
Lab 4: 0.0
Exam 1: 96-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 96.0
Lab 2 Letter: B
Lab sum: 217.0
_______________________________________
4 Student 4 Student Data
Lab 1: 0.0
Lab 2: 81.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 43-sent
Lab 5: 

## Algorithms

So far, we ended up with three different types of algorithms:

* `grade_calcuator` algorithms: Take grade from the student and return another grade. We use this for assigning letter grades and curving.
* `statistics_calculator` algorithms: Take a set of grades for all students and return numbers associated with the class. We used these to compute the mean and standard deviation for a class.
* `summary_calculator` algorithms: Take many grades from a student and return a another grade. We used these to sum grades of a student.

Here are the base classes:

In [ ]:
class grade_calculator(alg):    
    def __init__(self,name, stats):
        self.__stats=stats
        alg.__init__(self,name)

    def apply(self,a_grade):
        raise NotImplementedError
        # Returns a grade
    
class statistics_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,grades):
        raise NotImplementedError
        # returns numbers
                
class summary_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_student):
        raise NotImplementedError
        # returns a grade
        

While this approach works, users will need to be aware of how to use each type of algorithm and it would be difficult to create a system that would automate tasks, like curving a grade, that would require running several algorithms. We may be able to find a more general implementation that can unify all of these types of algorithms. There are several issues to consider:

* **Input**: Each of these algorithms take different inputs, either a single grade, all grades of a specific type for all students, or different grades for the same student.
* **Output**: Some of these algorithms generate new grades for each student, while others provide compute  over the whole class.
* **Information storage**: While we can store grades with students, other information like means and standard deviation have to do with the class and not any individual student. 

Lets first address the information storage issue. We simply need a mechanism to store information that has to do with a class instead of a student. The gradebook is what holds all of the students in the class, so we just add new data members to keep additional information. The only type of information that we are holding for now are specific exam means and standard deviations. We could choose to create new data objects for such information. Or we could simply stuff the information into a dictionary. The latter is easier and may suffice, so let's try it.



In [22]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class 
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
    
    def apply_summary(self,a_grader):
        for k,a_student in self.__students.items():
            a_student.add_grade(a_grader.apply(a_student))
    
    def apply_grader(self,a_grader,grade_name):
        for k,a_student in self.__students.items():
            a_student.add_grade(a_grader.apply(a_student[grade_name]))
            
    def apply_stats(self,a_stat_comp,grade_name):
        grades=list()
        for k,a_student in self.__students.items():
            grades.append(a_student[grade_name].value())
        return a_stat_comp.apply(grades)

    # Accessors
    
    def data(self):
        return self.__data

    def students(self):
        return self.__students
    
            
    def get_data(self,key=None):
        a_data=dict()
        for k,v in self.__data.items():
            if key:
                if key in k:
                    a_data[k]=v
            else:
                a_data[k]=v

        return a_data
    
    # Print functions
    def print_data(self):
        for k,v in self.__data.items():
            print (k,":",v)
 
    def print_grades(self,grade_name):
        if isinstance(grade_name,str):
            grade_names=list()
            grade_names.append(grade_name)
        else:
            grade_names=grade_name
                      
        for k,a_student in self.__students.items():
            print (a_student.name(),end="")
            for a_grade_name in grade_names:
                print (a_student[a_grade_name],end="")
            print()
            
    def print_students(self):    
        for k,a_student in self.__students.items():
            print (k, a_student.name())
            a_student.print_grades()
            print ("_______________________________________")
            

In [23]:
a_grade_book=grade_book("Test Gradebook")
a_grade_book["Example Data"]=100
print (a_grade_book["Example Data"])

100


Now lets address the input/output issue. We have several options:

1. Keep things the same. We keep 3 different types of algorithms. Note that necessarily needed to create separate apply methods in the `grade_book` class (`apply_grader`, `apply_stats`, ...) for each type of algorithm with a different method name. Here the user has to know what type of algorithm he/she is applying and then use the right apply method.

2. Keep the 3 different types of algorithms, but create a general apply method to `grade_book` that checks the type of the algorithm and then uses the right apply method.

3. Unify the 3 types of algorithms, pass into them the `grade_book` instance and let them decide what to do.

Option 2 is a reasonable solution, but let's try option 3 because it provides more flexibility for future algorithms that may take other input/output combinations. 

First turn our three different `calculator` algorithm base classes into one:

In [29]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError



Next, lets combine the `apply` methods in the `grade_book` class:

In [30]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class 
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
    

    # Accessors
    def data(self):
        return self.__data

    def students(self):
        return self.__students
    
            
    def get_data(self,key=None):
        a_data=dict()
        for k,v in self.__data.items():
            if key:
                if key in k:
                    a_data[k]=v
            else:
                a_data[k]=v

        return a_data
    
    # Print functions
    def print_data(self):
        for k,v in self.__data.items():
            print (k,":",v)
 
    def print_grades(self,grade_name):
        if isinstance(grade_name,str):
            grade_names=list()
            grade_names.append(grade_name)
        else:
            grade_names=grade_name
                      
        for k,a_student in self.__students.items():
            print (a_student.name(),end="")
            for a_grade_name in grade_names:
                print (a_student[a_grade_name],end="")
            print()
            
    def print_students(self):    
        for k,a_student in self.__students.items():
            print (k, a_student.name())
            a_student.print_grades()
            print ("_______________________________________")
            
            
    def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)



Now we have to move the logic of what each calculator algorithm uses into the algorithm. For example, consider the `uncurved_letter_grade_percent` algorithm. To run it we were using `apply_grader` method of the original `grade_book` class. Now, in order to use the `apply_calculator` method of the new `grade_book` class, we have to move the logic that iterates through students and applies and stores the grade from the old `apply_grader` method of `grade_book` to the new apply method of `uncurved_letter_grade_percent`.

In [31]:
# Code snippet of the old grade_book, for reference.

class grade_book_old(data):

    def apply_grader(self,a_grader,grade_name):
        for k,a_student in self.__students.items():
            a_student.add_grade(a_grader.apply(a_student[grade_name]))

# Code snippet of the old grader alg, for reference.
            
class uncurved_letter_grade_percent_old(grade_calculator):   

    def apply(self,a_grade):
        if not isinstance(a_grade,grade):
            print (self.name()+ " Error: Did not get an proper grade as input.")
            raise Exception
        if not a_grade.numerical():
            print (self.name()+ " Error: Did not get a numerical grade as input.")
            raise Exception

        percent=a_grade.value()/self.__max_grade
        
        for i,v in enumerate(self.__grades_definition):
            if percent>=v[0]:
                break
                            
        return grade(self.__grade_name,value=self.__grades_definition[i][1])
  

Let's make the modification:

In [32]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

Let's test with our class data:

In [33]:
# Rebuild the gradebook... with new grade_book class
a_grade_book=build_grade_book(df_0)

In [34]:
a_grade_book.apply_calculator(uncurved_letter_grade_percent("Lab 2",max_grade=100))


for k,a_student in a_grade_book.get_students().items():
    print (a_student.id_number(),a_student["Lab 2"],a_student["Lab 2 Letter"])

0 Lab 2: 0.0 Lab 2 Letter: F-
1 Lab 2: 96.0 Lab 2 Letter: A
2 Lab 2: 100.0 Lab 2 Letter: A+
3 Lab 2: 100.0 Lab 2 Letter: A+
4 Lab 2: 81.0 Lab 2 Letter: B-
5 Lab 2: 100.0 Lab 2 Letter: A+
6 Lab 2: 98.0 Lab 2 Letter: A+
7 Lab 2: 98.0 Lab 2 Letter: A+
8 Lab 2: 96.0 Lab 2 Letter: A
9 Lab 2: 78.0 Lab 2 Letter: C+
10 Lab 2: 76.0 Lab 2 Letter: C
11 Lab 2: 89.0 Lab 2 Letter: B+
12 Lab 2: 94.0 Lab 2 Letter: A
13 Lab 2: 77.0 Lab 2 Letter: C+
14 Lab 2: 96.0 Lab 2 Letter: A
15 Lab 2: 94.0 Lab 2 Letter: A


In [35]:
a_grade_book.print_students()

0 Student 0 Student Data
Lab 1: 0.0
Lab 2: 0.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 0.0
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 0.0
Lab 2 Letter: F-
_______________________________________
1 Student 1 Student Data
Lab 1: 100.0
Lab 2: 96.0
Lab 3: 46.0
Lab 4: 0.0
Exam 1: 68-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 68.0
Lab 2 Letter: A
_______________________________________
2 Student 2 Student Data
Lab 1: 100.0
Lab 2: 100.0
Lab 3: 90.0
Lab 4: 0.0
Exam 1: 88-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 88.0
Lab 2 Letter: A+
_______________________________________
3 Student 3 Student Data
Lab 1: 93.0
Lab 2: 100.0
Lab 3: 24.0
Lab 4: 0.0
Exam 1: 96-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 96.0
Lab 2 Letter: A+
_______________________________________
4 Student 4 Student Data
Lab 1: 0.0
Lab 2: 81.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 43-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed:

## Stats Computation

We have to perform a similar modification for all the other calculators. For example:

In [36]:
class mean_std_calculator(calculator):
    def __init__(self):
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name,**kwargs):
        grades=list()
        for k,a_student in a_grade_book.get_students().items():
            grades.append(a_student[grade_name].value())
        
        a_grade_book[grade_name+" Mean"] = np.mean(grades)
        a_grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))


In [38]:
a_grade_book.apply_calculator(mean_std_calculator(),grade_name="Lab 3")

print (a_grade_book["Lab 3 Mean"])
print (a_grade_book["Lab 3 STD"])

46.4375
36.617906736322325


## Next Steps

1. Migrate `grade_summer` and `curved_letter_grade`
1. remove n requirement from `grade_summer`
1. Update `mean_std_calculator`, adding cut_off and grade_name to constructor
1. Use `**kwargs` for passing overwrite
1. Add to grade_book
    1. get data
    1. print grades
    1. print data
1. Add curve function to gradebook
1. Create Letter Grade Counter Calculator
1. Create bell curve grade distributor

In [39]:
# 3

class mean_std_calculator(calculator):
    def __init__(self,grade_name,cut_off=None):
        self.__grade_name=grade_name
        self.__cut_off=cut_off
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name=None,cut_off=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
        if cut_off:
            pass
        else:
            cut_off=self.__cut_off
                    
        grades=list()
        for k,a_student in a_grade_book.get_students().items():
            a_grade_val=a_student[grade_name].value()
            if cut_off:
                if a_grade_val>cut_off:
                    grades.append(a_student[grade_name].value())
            else:
                grades.append(a_student[grade_name].value())
        
        a_grade_book[grade_name+" Mean"] = np.mean(grades)
        a_grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))
        a_grade_book[grade_name+" Max"] = max(grades)
        a_grade_book[grade_name+" Min"] = min(grades)

In [46]:
#1

class grade_summer(calculator):
    def __init__(self,prefix,n=None):
        self.__prefix=prefix
        self.__n=n
        calculator.__init__(self,"Sum Grades")
        
    def apply(self,a_gradebook,**kwargs):
        first=True
        
        for k,a_student in a_grade_book.get_students().items():
            if first:
                first=False                
                if self.__n:
                    labels=[self.__prefix+str(x) for x in range(1,self.__n)]
                else:
                    labels=list()
                    for i in range(1,1000):
                        label=self.__prefix+str(i)
                        try:
                            a_grade=a_student[label]
                            labels.append(label)
                        except:
                            break                

            grade_sum=0.
            for label in labels:
                grade_sum+=a_student[label].value()

            a_student.add_grade(grade(self.__prefix+"sum",value=grade_sum),**kwargs)

In [47]:
#1 

class curved_letter_grade(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        calculator.__init__(self,
                              "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                              " Mean="+str(self.__mean)+\
                              " STD="+str(self.__std)+\
                              " Max="+str(self.__max_grade))


    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name

        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception

            # Rescale the grade
            percent=a_grade.value()/self.__max_grade
            shift_to_zero=percent-(self.__mean/self.__max_grade)
            scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
            scaled_percent=scale_std+0.8

            for i,v in enumerate(self.__grades_definition):
                if scaled_percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]),**kwargs)
            

In [48]:
a_grade_book.apply_calculator(curved_letter_grade("Lab 3",
                                                  a_grade_book["Lab 3 Mean"],
                                                  a_grade_book["Lab 3 STD"]),
                              overwrite=True)

In [49]:
a_grade_book.print_students()

0 Student 0 Student Data
Lab 1: 0.0
Lab 2: 0.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 0.0
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 0.0
Lab 2 Letter: F-
Lab 3 Letter: C+
_______________________________________
1 Student 1 Student Data
Lab 1: 100.0
Lab 2: 96.0
Lab 3: 46.0
Lab 4: 52.5
Exam 1: 68-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 68.0
Lab 2 Letter: A
Lab 3 Letter: C+
_______________________________________
2 Student 2 Student Data
Lab 1: 100.0
Lab 2: 100.0
Lab 3: 90.0
Lab 4: 56.5
Exam 1: 88-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 88.0
Lab 2 Letter: A+
Lab 3 Letter: A-
_______________________________________
3 Student 3 Student Data
Lab 1: 93.0
Lab 2: 100.0
Lab 3: 24.0
Lab 4: 53.0
Exam 1: 96-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 96.0
Lab 2 Letter: A+
Lab 3 Letter: C
_______________________________________
4 Student 4 Student Data
Lab 1: 0.0
Lab 2: 81.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 4

## Running algorithms in sequence

Now that we have a common apply function for all calculators, we can easily abstract tasks that require several algorithms. For example:

In [40]:
algs=[# Sum the lab grades
      lambda: grade_summer("Lab ",n=5),  
    
      # Calculate the stats -> determine cut off
      lambda: mean_std_calculator("Lab sum",0.),
    
      # Calculate the stats with cut off
      #lambda: mean_std_calculator("Lab sum",a_grade_book["Lab sum Max"]/2.),
    
      # Curve using new stats
      lambda: curved_letter_grade("Lab sum",
                          a_grade_book["Lab sum Mean"],
                          a_grade_book["Lab sum STD"]) ]

In [41]:
algs

[<function __main__.<lambda>()>,
 <function __main__.<lambda>()>,
 <function __main__.<lambda>()>]

In [42]:
list(map(lambda x: a_grade_book.apply_calculator(x()), algs))

KeyError: 'Lab 1'

In [68]:
a_grade_book.print_students()

0 Student 0 Student Data
Lab 1: 0.0
Lab 2: 0.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 0.0
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 0.0
Lab sum: 0.0
Lab sum Letter: F-
_______________________________________
1 Student 1 Student Data
Lab 1: 100.0
Lab 2: 96.0
Lab 3: 46.0
Lab 4: 52.5
Exam 1: 68-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 68.0
Lab sum: 294.5
Lab sum Letter: C-
_______________________________________
2 Student 2 Student Data
Lab 1: 100.0
Lab 2: 100.0
Lab 3: 90.0
Lab 4: 56.5
Exam 1: 88-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 88.0
Lab sum: 346.5
Lab sum Letter: B
_______________________________________
3 Student 3 Student Data
Lab 1: 93.0
Lab 2: 100.0
Lab 3: 24.0
Lab 4: 53.0
Exam 1: 96-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 96.0
Lab sum: 270.0
Lab sum Letter: C+
_______________________________________
4 Student 4 Student Data
Lab 1: 0.0
Lab 2: 81.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 43

## Algorithm Classes
### Migrating Algorithms

We have redesigned the calculator part of our Grade Book framework. We now have to migrate the existing calculator algorithms to work with this redesign. The main thing to notice is that previously the `apply_xxx` methods

* found the data they needed in the grade book
* looped over that data
* applied the algorithm
* placed the results back into grade book

here are the methods of the old implementation:


In [ ]:
    def apply_summary(self,a_grader):
        for k,a_student in self.__students.items():
            a_student.add_grade(a_grader.apply(a_student))
    
    def apply_grader(self,a_grader,grade_name):
        for k,a_student in self.__students.items():
            a_student.add_grade(a_grader.apply(a_student[grade_name]))
            
    def apply_stats(self,a_stat_comp,grade_name):
        grades=list()
        for k,a_student in self.__students.items():
            grades.append(a_student[grade_name].value())
        return a_stat_comp.apply(grades)

which were replaced with:

In [ ]:
   def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)

Therefore, what ever was in the different `apply_xxx` functions of the old `grade_book` implementation have to be moved into the `apply` function of the calculators. 

For example compare the apply methods of the old and new implementation of this `grader` algorithm: 

In [ ]:
    # Old implementation
    def apply(self,a_grade):
        if not isinstance(a_grade,grade):
            print (self.name()+ " Error: Did not get an proper grade as input.")
            raise Exception
        if not a_grade.numerical():
            print (self.name()+ " Error: Did not get a numerical grade as input.")
            raise Exception
    
        # Rescale the grade
        percent=a_grade.value()/self.__max_grade
        shift_to_zero=percent-(self.__mean/self.__max_grade)
        scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
        scaled_percent=scale_std+0.8
        
        for i,v in enumerate(self.__grades_definition):
            if scaled_percent>=v[0]:
                break
                            
        return grade(self.__grade_name,value=self.__grades_definition[i][1])

In [9]:
    # migrated implementation
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

Let's test with our class data:

In [10]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError

class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

In [11]:
a_grade_book.apply_calculator(uncurved_letter_grade_percent("Lab 2",max_grade=100))
# for k,a_student in a_grade_book.get_students().items():
#     print (a_student.id_number(),a_student["Lab 2"],a_student["Lab 2 Letter"])

In [12]:
a_grade_book.print_students()

0 Student 0 Student Data
Lab 1: 0.0
Lab 2: 0.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 0.0
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 0.0
Lab 2 Letter: F-
_______________________________________
1 Student 1 Student Data
Lab 1: 100.0
Lab 2: 96.0
Lab 3: 46.0
Lab 4: 52.5
Exam 1: 68-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 68.0
Lab 2 Letter: A
_______________________________________
2 Student 2 Student Data
Lab 1: 100.0
Lab 2: 100.0
Lab 3: 90.0
Lab 4: 56.5
Exam 1: 88-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 88.0
Lab 2 Letter: A+
_______________________________________
3 Student 3 Student Data
Lab 1: 93.0
Lab 2: 100.0
Lab 3: 24.0
Lab 4: 53.0
Exam 1: 96-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 96.0
Lab 2 Letter: A+
_______________________________________
4 Student 4 Student Data
Lab 1: 0.0
Lab 2: 81.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 43-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fix

### Examples of other types of calculators

In [13]:
class mean_std_calculator(calculator):
    def __init__(self,grade_name,cut_off=None):
        self.__grade_name=grade_name
        self.__cut_off=cut_off
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name=None,cut_off=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
        if cut_off:
            pass
        else:
            cut_off=self.__cut_off
                    
        grades=list()
        for k,a_student in a_grade_book.get_students().items():
            a_grade_val=a_student[grade_name].value()
            if cut_off:
                if a_grade_val>cut_off:
                    grades.append(a_student[grade_name].value())
            else:
                grades.append(a_student[grade_name].value())
        
        a_grade_book[grade_name+" Mean"] = np.mean(grades)
        a_grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))
        a_grade_book[grade_name+" Max"] = max(grades)
        a_grade_book[grade_name+" Min"] = min(grades)

In [25]:
class grade_summer(calculator):
    def __init__(self,prefix,n=None):
        self.__prefix=prefix
        self.__n=n
        calculator.__init__(self,"Sum Grades")
        
    def apply(self,a_gradebook,**kwargs):
        first=True
        
        for k,a_student in a_grade_book.get_students().items():
            if first:
                first=False                
                if self.__n:
                    labels=[self.__prefix+str(x) for x in range(1,self.__n)]
                else:
                    labels=list()
                    for i in range(1,1000):
                        label=self.__prefix+str(i)
                        try:
                            a_grade=a_student[label]
                            labels.append(label)
                        except:
                            break                

            grade_sum=0.
            for label in labels:
                # Error print statement
                # print(label,a_student[label].value())
                grade_sum+=a_student[label].value()

            a_student.add_grade(grade(self.__prefix+"sum",value=grade_sum),**kwargs)

### Migration Exercise

Migrate the following algorithm from old to new calculator implementation. Use the example above to guide you. You will have to:

* Change the arguments of the apply function
* Insert loop over students (copy it from the analgous place above)
* Instead of returning the result, update the data within the grade book.

In [ ]:
class grade_calculator(alg):    
    def __init__(self,name, stats):
        self.__stats=stats
        alg.__init__(self,name)

    def apply(self,a_grade):
        raise NotImplementedError
        # Returns a grade
            
class curved_letter_grade(grade_calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        grade_calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))
        

    def apply(self,a_grade):
        if not isinstance(a_grade,grade):
            print (self.name()+ " Error: Did not get an proper grade as input.")
            raise Exception
        if not a_grade.numerical():
            print (self.name()+ " Error: Did not get a numerical grade as input.")
            raise Exception
    
        # Rescale the grade
        percent=a_grade.value()/self.__max_grade
        shift_to_zero=percent-(self.__mean/self.__max_grade)
        scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
        scaled_percent=scale_std+0.8
        
        for i,v in enumerate(self.__grades_definition):
            if scaled_percent>=v[0]:
                break
                            
        return grade(self.__grade_name,value=self.__grades_definition[i][1])
            

In [33]:
class curved_letter_grade(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))
        

    def apply(self,a_grade_book,grade_name=None,
              overwrite=False,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]

            if not isinstance(a_grade,grade):
                print (self.name()+ " Error: Did not get an proper grade as input.")
                raise Exception
            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception

            # Rescale the grade
            percent=a_grade.value()/self.__max_grade
            shift_to_zero=percent-(self.__mean/self.__max_grade)
            scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
            scaled_percent=scale_std+0.8

            for i,v in enumerate(self.__grades_definition):
                if scaled_percent>=v[0]:
                    break
                    
            # Add this grade back to the student 
            a_student.add_grade(grade(self.__grade_name+" Letter",value=self.__grades_definition[i][1]),
                                overwrite=overwrite)         

In [31]:
# Solution here
            
class curved_letter_grade(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        calculator.__init__(self,
                            "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                             " Mean="+str(self.__mean)+\
                             " STD="+str(self.__std)+\
                             " Max="+str(self.__max_grade))
        

    def apply(self,a_grade_book,grade_name=None,
              overwrite=False,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]
        
            if not isinstance(a_grade,grade):
                print (self.name()+ " Error: Did not get an proper grade as input.")
                raise Exception
            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception

            # Rescale the grade
            percent=a_grade.value()/self.__max_grade
            shift_to_zero=percent-(self.__mean/self.__max_grade)
            scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
            scaled_percent=scale_std+0.8

            for i,v in enumerate(self.__grades_definition):
                if scaled_percent>=v[0]:
                    break
                    
            #a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
                   
            a_student.add_grade(grade(self.__grade_name+" Letter",value=self.__grades_definition[i][1]),
                                overwrite=overwrite)



In [35]:
a_grade_book.apply_calculator(mean_std_calculator(grade_name="Lab 3"))
a_grade_book.apply_calculator(curved_letter_grade("Lab 3",
                                                  a_grade_book["Lab 3 Mean"],
                                                  a_grade_book["Lab 3 STD"]),
                                                  overwrite=True)

In [36]:
a_grade_book.print_students()

0 Student 0 Student Data
Lab 1: 0.0
Lab 2: 0.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 0.0
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 0.0
Lab 3 Letter: C+
_______________________________________
1 Student 1 Student Data
Lab 1: 100.0
Lab 2: 96.0
Lab 3: 46.0
Lab 4: 52.5
Exam 1: 68-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 68.0
Lab 3 Letter: C+
_______________________________________
2 Student 2 Student Data
Lab 1: 100.0
Lab 2: 100.0
Lab 3: 90.0
Lab 4: 56.5
Exam 1: 88-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 88.0
Lab 3 Letter: A-
_______________________________________
3 Student 3 Student Data
Lab 1: 93.0
Lab 2: 100.0
Lab 3: 24.0
Lab 4: 53.0
Exam 1: 96-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fixed: 96.0
Lab 3 Letter: C
_______________________________________
4 Student 4 Student Data
Lab 1: 0.0
Lab 2: 81.0
Lab 3: 0.0
Lab 4: 0.0
Exam 1: 43-sent
Lab 5: 0.0
Unnamed: 6: 0.0
Lab 7: 0.0
Exam 2: 0.0
Exam 1 Fix

## Running a sequence of algorithms



In [37]:
algs=[# Sum the lab grades
    lambda:
      grade_summer("Lab ",n=5),  
    
      # Calculate the stats -> determine cut off
    lambda:
      mean_std_calculator("Lab sum",0.),
    
      # Calculate the stats with cut off
      lambda :
          mean_std_calculator("Lab sum",a_grade_book["Lab sum Max"]/2.),
    
      # Curve using new stats
      lambda :
      curved_letter_grade("Lab sum",
                          a_grade_book["Lab sum Mean"],
                          a_grade_book["Lab sum STD"]) ]

In [38]:
list(map(lambda x: a_grade_book.apply_calculator(x(),overwrite=True), algs))

Lab 1 0.0
Lab 2 0.0
Lab 3 0.0
Lab 4 0.0
Lab 1 100.0
Lab 2 96.0
Lab 3 46.0
Lab 4 52.5
Lab 1 100.0
Lab 2 100.0
Lab 3 90.0
Lab 4 56.5
Lab 1 93.0
Lab 2 100.0
Lab 3 24.0
Lab 4 53.0
Lab 1 0.0
Lab 2 81.0
Lab 3 0.0
Lab 4 0.0
Lab 1 95.0
Lab 2 100.0
Lab 3 105.0
Lab 4 88.0
Lab 1 100.0
Lab 2 98.0
Lab 3 87.0
Lab 4 98.0
Lab 1 95.0
Lab 2 98.0
Lab 3 50.0
Lab 4 56.5
Lab 1 95.0
Lab 2 96.0
Lab 3 98.0
Lab 4 94.5
Lab 1 0.0
Lab 2 78.0
Lab 3 37.0
Lab 4 47.0
Lab 1 0.0
Lab 2 76.0
Lab 3 0.0
Lab 4 0.0
Lab 1 95.0
Lab 2 89.0
Lab 3 0.0
Lab 4 0.0
Lab 1 98.0
Lab 2 94.0
Lab 3 35.0
Lab 4 54.0
Lab 1 93.0
Lab 2 77.0
Lab 3 23.0
Lab 4 63.0
Lab 1 100.0
Lab 2 96.0
Lab 3 88.0
Lab 4 93.0
Lab 1 95.0
Lab 2 94.0
Lab 3 60.0
Lab 4 93.0


[None, None, None, None]

### More Exercises

1. Write an algorithm that removes grades that are less than a value (default 50% of max possible grade), and distributes grades as follows:
    * 16% A
    * 34% B
    * 34% C
    * 16% D

1. Write new algorithms:
    1. Write an algorithm that uses the grade boundries to assign grades, including +/-. 
    1. Write an algorithm that sums up a provided list of grades, dropping the lowest $n$.
    1. Write an algorithm that histograms letter grades, validating that it follows the bell curve distribution.

    For each:
        * Determine what type of algorithm.
        * Copy paste analogous algorithm.
        * Change name.
        * Remove code.
        * Add new code.
            * Sketch out how to implement the appropriate logic.
            * Perhaps work out specific pieces in another cell.
            * Implement.
        * Test.
             

    

In [39]:
class forced_curve(calculator):
    def __init__(self,grade_name,cut_off=None):
        self.__grade_name=grade_name
        self.__cut_off=cut_off
        calculator.__init__(self,"Forced Curve Boundry Statistical Calculator")
        
    def apply(self,a_grade_book,grade_name=None,cut_off=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
        if cut_off:
            pass
        else:
            cut_off=self.__cut_off
                    
        grades=list()
        for k,a_student in a_grade_book.get_students().items():
            a_grade_val=a_student[grade_name].value()
            if cut_off:
                if a_grade_val>cut_off:
                    grades.append(a_student[grade_name].value())
            else:
                grades.append(a_student[grade_name].value())
                
        # Possible Algorithm Logic
        # Sort the grades
        # Top 16% -> A
        # How many is 16%? Count all students (after min grade cut) -> .16 * n

        grades = sorted(grades)[::-1]
        n_students = len(grades)
        
        n_16 = int(0.16 * float(n_students))
        n_34 = int(0.34 * float(n_students))
        
        boundries= dict()
        boundries["A"]=grades[n_16-1]
        boundries["B"]=grades[n_16+n_34-1]
        boundries["C"]=grades[n_16+n_34+n_34-1]
        boundries["D"]=grades[min(n_students-1,n_16+n_34+n_34+n_16-1)]
        
        a_grade_book[grade_name+" Boundries"] = boundries


In [41]:
a_grade_book.apply_calculator(forced_curve(grade_name="Exam 1 Fixed"))



In [42]:
a_grade_book.print_data()

Lab 3 Mean : 46.4375
Lab 3 STD : 36.617906736322325
Lab 3 Max : 105.0
Lab 3 Min : 0.0
Lab sum Mean : 329.1818181818182
Lab sum STD : 47.97787024303863
Lab sum Max : 388.0
Lab sum Min : 256.0
Exam 1 Fixed Boundries : {'A': 98.0, 'B': 88.0, 'C': 64.0, 'D': 43.0}
